# Decoder model

## This model decodes EEG embeddings from another network to fMRI signal. This other network learns a space shared between EEG and fMRI.

In [1]:
import sys
sys.path.append("..")

import eeg_utils
import fmri_utils
import deep_cross_corr

import numpy as np
from numpy import correlate

import matplotlib.pyplot as plt

import mne
from nilearn.masking import apply_mask, compute_epi_mask

from sklearn.preprocessing import normalize

from scipy.signal import resample

import tensorflow.compat.v1 as tf

tf.enable_eager_execution()

import tensorflow.keras.backend as K

import sys

/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; i

### Load data

In [2]:
mask = fmri_utils.get_population_mask()

#reading data and spliting data into train and test by individuals
eeg_train, bold_train = deep_cross_corr.get_data(list(range(14)), masker=mask)
eeg_test, bold_test = deep_cross_corr.get_data(list(range(14, 16)), masker=mask)

eeg_train = eeg_train.reshape(eeg_train.shape[0], eeg_train.shape[1], eeg_train.shape[2], eeg_train.shape[3], 1)
eeg_test = eeg_test.reshape(eeg_test.shape[0], eeg_test.shape[1], eeg_test.shape[2], eeg_test.shape[3], 1)

bold_train = bold_train.reshape(bold_train.shape[0], bold_train.shape[1], bold_train.shape[2], 1)
bold_test = bold_test.reshape(bold_test.shape[0], bold_test.shape[1], bold_test.shape[2], 1)

/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarni

Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/32/export/20130410320002_Segmentation_bin.vhdr...
Setting channel info structure...
Reading 0 ... 162022  =      0.000 ...   648.088 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)


(16, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/35/export/20130424350002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 197234  =      0.000 ...   788.936 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)


...

(224, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/49/export/20130918490002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 167579  =      0.000 ...   670.316 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


(16, 14164, 20)
Extracting parameters from /home/davidcalhas/eeg_to_fmri/datasets/01/EEG/50/export/20131003_500002_Pulse_Artifact_Correction_bin.vhdr...
Setting channel info structure...
Reading 0 ... 168019  =      0.000 ...   672.076 secs...


../eeg_utils.py:35: RuntimeWarning: No coordinate information found for channels ['ECG', 'O9', 'O10']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  return mne.io.read_raw_brainvision(complete_path, preload=True)
/home/davidcalhas/anaconda3/envs/fmri_eeg/lib/python3.6/site-packages/nilearn/image/resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


(32, 14164, 20)


#### So the encoder and decoder model are being trained separately, let's see if we train them at the same time, and propagate the loss of the decoder to the enconder, the performance increases

# Encoder and Decoder training

In [3]:
eeg_input_shape = (eeg_train.shape[1], eeg_train.shape[2], eeg_train.shape[3], 1)
kernel_size = (eeg_train.shape[1], eeg_train.shape[2], 1)
eeg_network = deep_cross_corr.eeg_network(eeg_input_shape, kernel_size)
print(eeg_network.summary())



bold_input_shape = (bold_train.shape[1], bold_train.shape[2], 1)
kernel_size = (bold_train.shape[1], 1)
bold_network = deep_cross_corr.bold_network(bold_input_shape, kernel_size)
print(bold_network.summary())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 63, 2, 20, 1)      5         
_________________________________________________________________
batch_normalization (BatchNo (None, 63, 2, 20, 1)      4         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 62, 1, 20, 1)      5         
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 1, 20, 1)      4         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 61, 1, 20, 1)      3         
_________________________________________________________________
batch_normalization_2 (Batch (None, 61, 1, 20, 1)      4         
____________________________

In [4]:
input_eeg = tf.keras.layers.Input(shape=eeg_input_shape)
input_bold = tf.keras.layers.Input(shape=bold_input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_eeg = eeg_network(input_eeg)
processed_bold = bold_network(input_bold)

correlation = tf.keras.layers.Lambda(deep_cross_corr.correlation, 
                     output_shape=deep_cross_corr.cos_dist_output_shape, name="correlation_layer")([processed_eeg, processed_bold])

multi_modal_model = tf.keras.Model([input_eeg, input_bold], correlation)

#alpha = 0.5
#corr_loss = deep_cross_corr.contrastive_loss
#decod_loss = deep_cross_corr.correlation
#multi_modal_model.compile(loss={"correlation_layer":corr_loss, "decoding_layer": decod_loss], loss_weights=[alpha, 1-alpha], optimizer=Adam(lr=0.0001))

### Load activations from compression network

In [5]:
shared_eeg_train = eeg_network.predict(eeg_train)
shared_eeg_test = eeg_network.predict(eeg_test)

### Build the decoding neural network architecture

In [6]:
input_shape = (None, shared_eeg_train.shape[1], shared_eeg_train.shape[2], 1)

decoder_model = tf.keras.Sequential([
    tf.keras.layers.Conv2DTranspose(1, kernel_size=(100, 1),
                          activation='selu', strides=(3,1), input_shape=input_shape[1:]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=(100, 1), 
                          activation='selu', strides=(50,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ZeroPadding2D(padding=(57,0))
], name="bold_synthesizer")

decoder_model.build(input_shape)

print(decoder_model.summary())

Model: "bold_synthesizer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 280, 20, 1)        101       
_________________________________________________________________
batch_normalization_5 (Batch (None, 280, 20, 1)        4         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14050, 20, 1)      101       
_________________________________________________________________
batch_normalization_6 (Batch (None, 14050, 20, 1)      4         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 14164, 20, 1)      0         
Total params: 210
Trainable params: 206
Non-trainable params: 4
_________________________________________________________________
None


In [7]:
#decoder_model.compile(optimizer='adam', loss=deep_cross_corr.cross_correlation)

### Let's train the model

In [8]:
#model.fit(shared_eeg_train, bold_train, epochs=100)

In [9]:
#correlation_decoder = Model(inputs=[input_eeg, input_bold], outputs=[correlation, last_layer_decoder])

In [10]:
#alpha = 0.5
#corr_loss = deep_cross_corr.contrastive_loss
#decod_loss = deep_cross_corr.correlation_decoder_loss
#correlation_decoder.compile(loss=[corr_loss, decod_loss], loss_weights=[1, 1], optimizer=Adam(lr=0.0001))

In [11]:
X_train_eeg, X_train_bold, tr_y = deep_cross_corr.create_eeg_bold_pairs(eeg_train, bold_train)
X_test_eeg, X_test_bold, te_y = deep_cross_corr.create_eeg_bold_pairs(eeg_test, bold_test)

In [12]:
#correlation_decoder.fit([X_train_eeg, X_train_bold], [tr_y, X_train_bold], 
#                       epochs=20)

### The model is not training as supposed, so a custom training is needed

### Implement the customized training loop

In [13]:
X_train_eeg = tf.convert_to_tensor(X_train_eeg, dtype=np.float32)
X_train_bold = tf.convert_to_tensor(X_train_bold, dtype=np.float32)
tr_y = tf.convert_to_tensor(tr_y, dtype=np.float32)

In [14]:
def loss_decoder(model, inputs, targets):
    reconstruction_loss = deep_cross_corr.cross_correlation(outputs, targets)
    return K.mean(reconstruction_loss)


def grad_decoder(model, inputs, targets):
    with tf.GradientTape() as tape:    
        tape.watch(inputs)
        outputs = model(inputs)
        
        #loss
        reconstruction_loss = deep_cross_corr.cross_correlation(outputs, targets)
        reconstruction_loss = K.mean(reconstruction_loss)
        return -reconstruction_loss,  tape.gradient(-reconstruction_loss, model.trainable_weights)

In [15]:
def grad_multi_encoder(model, inputs, targets, reconstruction_loss):
    with tf.GradientTape() as tape:    
        tape.watch(inputs)
        outputs = model(inputs)
        
        #loss
        encoder_loss = abs(deep_cross_corr.contrastive_loss(outputs, targets)) + abs(reconstruction_loss)
        return -encoder_loss,  tape.gradient(-encoder_loss, 
                                            model.trainable_weights)

In [16]:
class custom_training_loss:
    def __init__(self):
        self.encoder_loss = 0
        self.decoder_loss = 0
        self.batch = 0
        
    def update_batch_decoder_loss_avg(self, loss):
        self.decoder_loss += loss
        self.batch += 1
    
    def get_batch_decoder_loss_avg(self):
        return self.decoder_loss/self.batch
    
    def update_batch_encoder_loss_avg(self, loss):
        self.encoder_loss += loss
        self.batch += 1
    
    def get_batch_encoder_loss_avg(self):
        return self.encoder_loss/self.batch

In [17]:
# keep results for plotting
train_loss_results = []
train_accuracy_results = []

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

global_step = tf.Variable(0)

num_epochs = 10

for epoch in range(num_epochs):
    
    losses = custom_training_loss()
    # Training loop - using batches of 32
    batch_size = 128
    for batch_init in range(0, len(X_train_eeg), batch_size):
        batch_start = batch_init
        if(batch_start + batch_size >= len(X_train_eeg)):
            batch_stop = len(X_train_eeg)
        else:
            batch_stop = batch_start + batch_size
        
        shared_eeg = eeg_network(X_train_eeg[batch_start:batch_stop])
        
        # Optimize the synthesizer model
        decoder_loss, decoder_grads = grad_decoder(decoder_model, shared_eeg, X_train_bold[batch_start:batch_stop])
        optimizer.apply_gradients(zip(decoder_grads, decoder_model.trainable_variables), 
                                  global_step)

        
        #now train the compression by correlation model
        encoder_loss, encoder_grads = grad_multi_encoder(multi_modal_model, 
                                                         [X_train_eeg[batch_start:batch_stop], 
                                                                             X_train_bold[batch_start:batch_stop]], 
                                                         tr_y[batch_start:batch_stop], decoder_loss)
        optimizer.apply_gradients(zip(encoder_grads, multi_modal_model.trainable_variables), 
                                  global_step)
        # Track progress
        losses.update_batch_decoder_loss_avg(decoder_loss)
        losses.update_batch_encoder_loss_avg(encoder_loss)

    # end epoch
    decoder_loss = losses.get_batch_decoder_loss_avg()
    encoder_loss = losses.get_batch_encoder_loss_avg()
    
    print(encoder_loss, decoder_loss)
    #if epoch % 50 == 0:
    #    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, 
    #                                                                epoch_loss_avg.result(), 
    #                                                                epoch_accuracy.result()))


KeyboardInterrupt: 